In [9]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import KNNBasic
import os
import pandas as pd
from surprise import SVD
from surprise.model_selection import train_test_split

user = pd.read_csv('BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1")
user.columns = ['userID', 'Location', 'Age']
rating = pd.read_csv('BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")
rating.columns = ['userID', 'ISBN', 'bookRating']
df = pd.merge(user, rating, on='userID', how='inner')
df.drop(['Location', 'Age'], axis=1, inplace=True)
df.head()

"""
User 정보와 Book Rating 정보가 담긴 csv 파일을 읽어서 데이터 프레임에 merge
그 후에 Location, Age 열을 삭제
--> 유저, 아이템, 레이팅 자료구조 획득
"""

'\nUser 정보와 Book Rating 정보가 담긴 csv 파일을 읽어서 데이터 프레임에 merge\n그 후에 Location, Age 열을 삭제\n--> 유저, 아이템, 레이팅 자료구조 획득\n'

In [10]:
#원본 데이터 출력
df.head()

,userID,ISBN,bookRating
0,2,0195153448,0
1,7,034542252,0
2,8,0002005018,5
3,8,0060973129,0
4,8,0374157065,0


In [25]:
#데이터가 너무 크기 때문에 user를 400여명 정도로 변경
df = df.iloc[0:3000,]

df.head()

,userID,ISBN,bookRating
0,2,0195153448,0
1,7,034542252,0
2,8,0002005018,5
3,8,0060973129,0
4,8,0374157065,0


In [26]:
#현재 자료구조 내에 있는 유저 출력
print(df['userID'].nunique())
df['userID'].unique()

431


array([   2,    7,    8,    9,   10,   12,   14,   16,   17,   19,   20,
         22,   23,   26,   32,   36,   38,   39,   42,   44,   51,   53,
         56,   64,   67,   68,   69,   70,   73,   75,   77,   78,   79,
         81,   82,   83,   85,   86,   87,   88,   91,   92,   95,   97,
         99,  100,  102,  107,  109,  110,  114,  125,  129,  132,  133,
        135,  137,  139,  141,  144,  151,  160,  162,  165,  169,  176,
        178,  182,  183,  185,  190,  193,  199,  202,  204,  207,  212,
        215,  217,  221,  224,  226,  228,  230,  232,  233,  236,  237,
        241,  242,  243,  244,  247,  249,  250,  254,  256,  257,  269,
        272,  273,  278,  280,  289,  291,  300,  302,  306,  310,  311,
        313,  316,  323,  326,  327,  332,  334,  337,  338,  339,  343,
        345,  356,  357,  359,  361,  362,  364,  367,  369,  372,  376,
        382,  383,  384,  387,  388,  392,  393,  400,  406,  408,  413,
        422,  424,  430,  431,  432,  433,  435,  4

In [27]:
#surprise 라이브러리를 이용하여 data로드
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(df[['userID', 'ISBN', 'bookRating']], reader)

In [28]:
#data를 train과 test로 나누고 알고리즘 적용 후 fit 함수 적용
#KNNBasic 알고리즘 적용
trainset, testset = train_test_split(data, test_size=0.25)
sim_options = {'name' : 'cosine', 'user_based' : True}
algo = KNNBasic(k = 40, min_k = 1, sim_options = sim_options)

algo.fit(trainset)

#유저별 아이템간 유사도 측정
uids = df['userID'].unique()
for i in range(1, len(uids)):
    uid = uids[i-1]
    iids = df[df.userID == uid]
    for j in range(1, len(iids)):
        iid = iids[j-1 : j].ISBN.values[0]
        r_ui = iids[j-1 : j].bookRating.values[0]
        pred = algo.predict(uid, iid, r_ui, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
user: 8          item: 0002005018 r_ui = 5.00   est = 5.00   {'actual_k': 1, 'was_impossible': False}
user: 8          item: 0060973129 r_ui = 0.00   est = 3.76   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 8          item: 0374157065 r_ui = 0.00   est = 3.76   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 8          item: 0393045218 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 8          item: 0399135782 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 8          item: 0425176428 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 8          item: 0671870432 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 8          item: 0679425608 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 8          item: 074322678X r_ui = 5.00   est = 5.00

C:\Users\user\Anaconda3\lib\site-packages\surprise\prediction_algorithms\algo_base.py:248: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)



user: 183        item: 9726116384 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 183        item: 9726593855 r_ui = 9.00   est = 3.76   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 183        item: 9726621445 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 183        item: 9726621720 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 183        item: 9726626536 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 183        item: 9726650178 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 183        item: 9726652499 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 183        item: 9726653525 r_ui = 9.00   est = 9.00   {'actual_k': 1, 'was_impossible': False}
user: 183        item: 9726953308 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 183        item: 9726991366 r_ui = 8.00   est = 8.

user: 254        item: 0373835582 r_ui = 0.00   est = 3.76   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 254        item: 0373835590 r_ui = 0.00   est = 3.76   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 254        item: 0373836023 r_ui = 7.00   est = 3.76   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 254        item: 0375503269 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 254        item: 0375503838 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 254        item: 0375713549 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 254        item: 0375724370 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 254        item: 0375727345 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 254        item: 0375751513 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 2

user: 388        item: 0553569058 r_ui = 10.00   est = 10.00   {'actual_k': 1, 'was_impossible': False}
user: 388        item: 0679427430 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 388        item: 0679725393 r_ui = 9.00   est = 9.00   {'actual_k': 1, 'was_impossible': False}
user: 388        item: 0802114369 r_ui = 8.00   est = 3.76   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 388        item: 0803233019 r_ui = 0.00   est = 3.76   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 388        item: 0819154946 r_ui = 8.00   est = 3.76   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 388        item: 0897322339 r_ui = 7.00   est = 3.76   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 388        item: 0940322978 r_ui = 4.00   est = 3.76   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 388        item: 1573225126 r_ui = 5.00   est = 5.0

user: 507        item: 067144378X r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 507        item: 067151699X r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 507        item: 0671885634 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 507        item: 0679833692 r_ui = 0.00   est = 3.76   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 507        item: 0679893105 r_ui = 9.00   est = 9.00   {'actual_k': 1, 'was_impossible': False}
user: 507        item: 0684196387 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 507        item: 068484267X r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 507        item: 068815252X r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 507        item: 0688169678 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 507        item: 0689710879 r_ui = 10.00   est = 3.

user: 651        item: 0962893862 r_ui = 4.00   est = 4.00   {'actual_k': 1, 'was_impossible': False}
user: 651        item: 1555834531 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 651        item: 1555835953 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 651        item: 156341029X r_ui = 8.00   est = 3.76   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 651        item: 1563410443 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 651        item: 1563411148 r_ui = 6.00   est = 6.00   {'actual_k': 1, 'was_impossible': False}
user: 651        item: 1563899388 r_ui = 6.00   est = 6.00   {'actual_k': 1, 'was_impossible': False}
user: 651        item: 1886383421 r_ui = 4.00   est = 4.00   {'actual_k': 1, 'was_impossible': False}
user: 657        item: 0345429176 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 657        item: 038529929X r_ui = 6.00   est = 6.0

user: 850        item: 3442248922 r_ui = 0.00   est = 3.76   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 850        item: 3453016416 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 850        item: 3453046196 r_ui = 4.00   est = 4.00   {'actual_k': 1, 'was_impossible': False}
user: 850        item: 345309221X r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 850        item: 3453106520 r_ui = 0.00   est = 3.76   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 850        item: 3453137205 r_ui = 7.00   est = 7.00   {'actual_k': 1, 'was_impossible': False}
user: 850        item: 3453147243 r_ui = 5.00   est = 5.00   {'actual_k': 1, 'was_impossible': False}
user: 850        item: 3453171977 r_ui = 10.00   est = 10.00   {'actual_k': 1, 'was_impossible': False}
user: 850        item: 3453177800 r_ui = 5.00   est = 5.00   {'actual_k': 1, 'was_impossible': False}
user: 850        item: 3471791

user: 989        item: 0316825336 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 989        item: 0374216495 r_ui = 0.00   est = 3.76   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 989        item: 039914739X r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 989        item: 0679419152 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 989        item: 0786867086 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 989        item: 0871137631 r_ui = 0.00   est = 3.76   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 996        item: 0060192895 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 996        item: 0140259198 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 996        item: 0345435249 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 998        item: 226602665

user: 1211       item: 0006513204 r_ui = 7.00   est = 7.00   {'actual_k': 1, 'was_impossible': False}
user: 1211       item: 0006513409 r_ui = 0.00   est = 3.76   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 1211       item: 0099771519 r_ui = 9.00   est = 9.00   {'actual_k': 2, 'was_impossible': False}
user: 1211       item: 0140274294 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 1211       item: 0140621725 r_ui = 8.00   est = 8.00   {'actual_k': 1, 'was_impossible': False}
user: 1211       item: 014143984X r_ui = 9.00   est = 9.00   {'actual_k': 1, 'was_impossible': False}
user: 1211       item: 0307177092 r_ui = 0.00   est = 3.76   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 1211       item: 0330367358 r_ui = 8.00   est = 8.00   {'actual_k': 1, 'was_impossible': False}
user: 1211       item: 0330375253 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 1211       item: 037572632

In [29]:
#KNNWithMeans 알고리즘 적용

from surprise import KNNWithMeans

trainset, testset = train_test_split(data, test_size=0.25)
sim_options = {'name' : 'cosine', 'user_based' : True}
algo = KNNWithMeans(k = 40, min_k = 1, sim_options = sim_options)

algo.fit(trainset)

uids = df['userID'].unique()
###
for i in range(1, len(uids)):
    uid = uids[i-1]
    iids = df[df.userID == uid]
    for j in range(1, len(iids)):
        iid = iids[j-1 : j].ISBN.values[0]
        r_ui = iids[j-1 : j].bookRating.values[0]
        pred = algo.predict(uid, iid, r_ui, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
user: 8          item: 0002005018 r_ui = 5.00   est = 5.00   {'actual_k': 1, 'was_impossible': False}
user: 8          item: 0060973129 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 8          item: 0374157065 r_ui = 0.00   est = 3.80   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 8          item: 0393045218 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 8          item: 0399135782 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 8          item: 0425176428 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 8          item: 0671870432 r_ui = 0.00   est = 3.80   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 8          item: 0679425608 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 8          item: 074322678X r_ui = 5.00   est = 5.00

user: 183        item: 9728440693 r_ui = 7.00   est = 7.00   {'actual_k': 1, 'was_impossible': False}
user: 183        item: 9728440804 r_ui = 9.00   est = 9.00   {'actual_k': 1, 'was_impossible': False}
user: 183        item: 9728487002 r_ui = 10.00   est = 10.00   {'actual_k': 1, 'was_impossible': False}
user: 183        item: 9728487142 r_ui = 10.00   est = 10.00   {'actual_k': 1, 'was_impossible': False}
user: 183        item: 9728487150 r_ui = 8.00   est = 8.00   {'actual_k': 1, 'was_impossible': False}
user: 183        item: 9728487266 r_ui = 6.00   est = 6.00   {'actual_k': 1, 'was_impossible': False}
user: 183        item: 9728487304 r_ui = 6.00   est = 6.00   {'actual_k': 1, 'was_impossible': False}
user: 183        item: 9728487355 r_ui = 7.00   est = 7.00   {'actual_k': 1, 'was_impossible': False}
user: 183        item: 9728487576 r_ui = 6.00   est = 6.00   {'actual_k': 1, 'was_impossible': False}
user: 183        item: 9728487606 r_ui = 6.00   est = 6.00   {'actual_k': 1, '

user: 254        item: 0590402129 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 254        item: 059043893X r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 254        item: 0590440675 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 254        item: 0613028155 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 254        item: 0618002219 r_ui = 9.00   est = 9.00   {'actual_k': 1, 'was_impossible': False}
user: 254        item: 0618002227 r_ui = 8.00   est = 8.00   {'actual_k': 1, 'was_impossible': False}
user: 254        item: 0618002235 r_ui = 0.00   est = 3.80   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 254        item: 0618002243 r_ui = 0.00   est = 3.80   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 254        item: 0670035262 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 254        item: 067101531

user: 472        item: 080521061X r_ui = 9.00   est = 3.80   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 472        item: 0823032612 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 472        item: 0865732434 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 476        item: 0312288115 r_ui = 5.00   est = 5.00   {'actual_k': 1, 'was_impossible': False}
user: 476        item: 0316171107 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 476        item: 0380710218 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 476        item: 0515118656 r_ui = 8.00   est = 8.00   {'actual_k': 1, 'was_impossible': False}
user: 476        item: 0553289063 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 476        item: 0671000306 r_ui = 0.00   est = 3.80   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 476        item: 067167368

user: 638        item: 0140128107 r_ui = 10.00   est = 10.00   {'actual_k': 1, 'was_impossible': False}
user: 638        item: 0151000719 r_ui = 10.00   est = 10.00   {'actual_k': 1, 'was_impossible': False}
user: 638        item: 0151001006 r_ui = 10.00   est = 3.80   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 638        item: 0316603287 r_ui = 9.00   est = 9.00   {'actual_k': 1, 'was_impossible': False}
user: 638        item: 0316666343 r_ui = 10.00   est = 6.40   {'actual_k': 3, 'was_impossible': False}
user: 638        item: 0316693006 r_ui = 10.00   est = 8.34   {'actual_k': 2, 'was_impossible': False}
user: 638        item: 0316693200 r_ui = 9.00   est = 9.00   {'actual_k': 1, 'was_impossible': False}
user: 638        item: 0316769487 r_ui = 10.00   est = 8.60   {'actual_k': 2, 'was_impossible': False}
user: 638        item: 0316776963 r_ui = 10.00   est = 10.00   {'actual_k': 2, 'was_impossible': False}
user: 638        item: 0316779423 r_ui = 10.00 

user: 741        item: 0553561618 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 741        item: 0553581279 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 741        item: 0590431250 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 741        item: 0590514776 r_ui = 6.00   est = 3.80   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 741        item: 061312975X r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 741        item: 068983571X r_ui = 0.00   est = 3.80   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 741        item: 0743201078 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 741        item: 07472668   r_ui = 10.00   est = 10.00   {'actual_k': 1, 'was_impossible': False}
user: 741        item: 0765341972 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 741        item: 0812504

user: 882        item: 0553569058 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 882        item: 0553569074 r_ui = 0.00   est = 3.80   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 882        item: 0553571656 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 882        item: 0553572997 r_ui = 10.00   est = 10.00   {'actual_k': 1, 'was_impossible': False}
user: 882        item: 0553801945 r_ui = 0.00   est = 3.80   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 882        item: 0590445561 r_ui = 0.00   est = 3.80   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 882        item: 0618260307 r_ui = 10.00   est = 10.00   {'actual_k': 1, 'was_impossible': False}
user: 882        item: 067081976X r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 882        item: 0670869902 r_ui = 0.00   est = 3.80   {'was_impossible': True, 'reason': 'User and

user: 1025       item: 0750003049 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 1025       item: 0752853007 r_ui = 6.00   est = 3.80   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 1025       item: 0756401453 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 1025       item: 0805067868 r_ui = 10.00   est = 3.80   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 1025       item: 0811828379 r_ui = 9.00   est = 9.00   {'actual_k': 1, 'was_impossible': False}
user: 1025       item: 0855030224 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 1025       item: 0860079481 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 1025       item: 0861638522 r_ui = 5.00   est = 5.00   {'actual_k': 1, 'was_impossible': False}
user: 1025       item: 0862648092 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 1025       item: 09632707

user: 1248       item: 0805051732 r_ui = 9.00   est = 9.00   {'actual_k': 1, 'was_impossible': False}
user: 1248       item: 0811807150 r_ui = 9.00   est = 9.00   {'actual_k': 1, 'was_impossible': False}
user: 1248       item: 0934257345 r_ui = 9.00   est = 3.80   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 1248       item: 1558534245 r_ui = 10.00   est = 10.00   {'actual_k': 1, 'was_impossible': False}
user: 1248       item: 1560980087 r_ui = 10.00   est = 3.80   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 1249       item: 0440226430 r_ui = 5.00   est = 5.00   {'actual_k': 1, 'was_impossible': False}
user: 1249       item: 0449300021 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 1249       item: 0671004565 r_ui = 8.00   est = 5.00   {'actual_k': 0, 'was_impossible': False}
user: 1249       item: 0684835983 r_ui = 8.00   est = 5.00   {'actual_k': 0, 'was_impossible': False}


In [30]:
#KNNWithZScore 알고리즘 적용
from surprise import KNNWithZScore

trainset, testset = train_test_split(data, test_size=0.25)
sim_options = {'name' : 'cosine', 'user_based' : True}
algo = KNNWithZScore(k = 40, min_k = 1, sim_options = sim_options)

algo.fit(trainset)

uids = df['userID'].unique()
###
for i in range(1, len(uids)):
    uid = uids[i-1]
    iids = df[df.userID == uid]
    for j in range(1, len(iids)):
        iid = iids[j-1 : j].ISBN.values[0]
        r_ui = iids[j-1 : j].bookRating.values[0]
        pred = algo.predict(uid, iid, r_ui, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
user: 8          item: 0002005018 r_ui = 5.00   est = 5.00   {'actual_k': 1, 'was_impossible': False}
user: 8          item: 0060973129 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 8          item: 0374157065 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 8          item: 0393045218 r_ui = 0.00   est = 3.84   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 8          item: 0399135782 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 8          item: 0425176428 r_ui = 0.00   est = 3.84   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 8          item: 0671870432 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 8          item: 0679425608 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 8          item: 074322678X r_ui = 5.00   est = 5.00

user: 183        item: 9729817715 r_ui = 6.00   est = 6.00   {'actual_k': 1, 'was_impossible': False}
user: 183        item: 9729852405 r_ui = 8.00   est = 8.00   {'actual_k': 1, 'was_impossible': False}
user: 185        item: 0380778556 r_ui = 10.00   est = 3.84   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 199        item: 0373196407 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 199        item: 0373196520 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 199        item: 0385336772 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 199        item: 0440949424 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 204        item: 0451452755 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 228        item: 0385337639 r_ui = 7.00   est = 7.00   {'actual_k': 1, 'was_impossible': False}
user: 232        item: 0007123817 r_ui = 7.00   est = 7.

user: 254        item: 0590440675 r_ui = 0.00   est = 3.84   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 254        item: 0613028155 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 254        item: 0618002219 r_ui = 9.00   est = 9.00   {'actual_k': 1, 'was_impossible': False}
user: 254        item: 0618002227 r_ui = 8.00   est = 3.84   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 254        item: 0618002235 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 254        item: 0618002243 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 254        item: 0670035262 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 254        item: 0671015311 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 254        item: 0671021001 r_ui = 7.00   est = 1.53   {'actual_k': 0, 'was_impossible': False}
user: 254        item: 067104048

user: 446        item: 0486292940 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 446        item: 051762639X r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 446        item: 0553145460 r_ui = 10.00   est = 10.00   {'actual_k': 1, 'was_impossible': False}
user: 446        item: 0684824442 r_ui = 9.00   est = 9.00   {'actual_k': 1, 'was_impossible': False}
user: 446        item: 0743458303 r_ui = 0.00   est = 3.84   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 446        item: 081296666X r_ui = 5.00   est = 5.00   {'actual_k': 1, 'was_impossible': False}
user: 446        item: 0819215635 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 446        item: 0880011025 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 446        item: 1561011525 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 460        item: 00265124   r_ui = 0.00   est = 0

user: 626        item: 3499222604 r_ui = 0.00   est = 3.84   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 626        item: 3502670005 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 626        item: 3505093491 r_ui = 0.00   est = 3.84   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 626        item: 3505094331 r_ui = 0.00   est = 3.84   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 626        item: 350509434X r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 626        item: 3505094374 r_ui = 0.00   est = 3.84   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 626        item: 3596126827 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 626        item: 3596237262 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 626        item: 370662270X r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_im

user: 741        item: 0373123167 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 741        item: 0373123205 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 741        item: 0373123248 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 741        item: 0373123485 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 741        item: 037316971X r_ui = 0.00   est = 3.84   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 741        item: 0373196679 r_ui = 0.00   est = 3.84   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 741        item: 0373196725 r_ui = 10.00   est = 10.00   {'actual_k': 1, 'was_impossible': False}
user: 741        item: 0373196741 r_ui = 3.00   est = 3.84   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 741        item: 0373245696 r_ui = 0.00   est = 3.84   {'was_impossible': True, 'reason': 'User and/o

user: 882        item: 1904301088 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 885        item: 015644450X r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 885        item: 0380002450 r_ui = 9.00   est = 0.00   {'actual_k': 0, 'was_impossible': False}
user: 896        item: 0140620338 r_ui = 8.00   est = 3.84   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 899        item: 0971880107 r_ui = 2.00   est = 2.00   {'actual_k': 1, 'was_impossible': False}
user: 899        item: 3257061269 r_ui = 9.00   est = 3.84   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 899        item: 3404126661 r_ui = 0.00   est = 3.84   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 899        item: 3453088433 r_ui = 4.00   est = 4.00   {'actual_k': 1, 'was_impossible': False}
user: 899        item: 3453177622 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 8

user: 1095       item: 0865532168 r_ui = 10.00   est = 3.84   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 1095       item: 3478088534 r_ui = 5.00   est = 5.00   {'actual_k': 1, 'was_impossible': False}
user: 1096       item: 059040640X r_ui = 7.00   est = 7.00   {'actual_k': 1, 'was_impossible': False}
user: 1098       item: 031227999X r_ui = 7.00   est = 7.00   {'actual_k': 1, 'was_impossible': False}
user: 1111       item: 0316666343 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 1113       item: 0307132668 r_ui = 0.00   est = 3.84   {'was_impossible': True, 'reason': 'User and/or item is unkown.'}
user: 1113       item: 0373169876 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 1113       item: 0380723638 r_ui = 0.00   est = 0.00   {'actual_k': 1, 'was_impossible': False}
user: 1113       item: 0385324057 r_ui = 9.00   est = 9.00   {'actual_k': 1, 'was_impossible': False}
user: 1113       item: 06710210

In [31]:
#SVD 알고리즘 적용
from surprise import SVD

trainset, testset = train_test_split(data, test_size=0.25)
algo = SVD(n_factors = 100, n_epochs = 20, biased = False, lr_all = 0.005, reg_all = 0)

algo.fit(trainset)

uids = df['userID'].unique()
###
for i in range(1, len(uids)):
    uid = uids[i-1]
    iids = df[df.userID == uid]
    for j in range(1, len(iids)):
        iid = iids[j-1 : j].ISBN.values[0]
        r_ui = iids[j-1 : j].bookRating.values[0]
        pred = algo.predict(uid, iid, r_ui, verbose=True)

user: 8          item: 0002005018 r_ui = 5.00   est = 1.62   {'was_impossible': False}
user: 8          item: 0060973129 r_ui = 0.00   est = 3.81   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 8          item: 0374157065 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 8          item: 0393045218 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 8          item: 0399135782 r_ui = 0.00   est = 0.02   {'was_impossible': False}
user: 8          item: 0425176428 r_ui = 0.00   est = 3.81   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 8          item: 0671870432 r_ui = 0.00   est = 3.81   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 8          item: 0679425608 r_ui = 0.00   est = 0.28   {'was_impossible': False}
user: 8          item: 074322678X r_ui = 5.00   est = 3.81   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 8          item: 0771025661 r_ui = 0.00   est = 3.81   {'wa

user: 242        item: 3257207522 r_ui = 10.00   est = 9.89   {'was_impossible': False}
user: 242        item: 3257208626 r_ui = 8.00   est = 7.99   {'was_impossible': False}
user: 242        item: 3257208634 r_ui = 10.00   est = 3.81   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 242        item: 3257208640 r_ui = 9.00   est = 9.00   {'was_impossible': False}
user: 242        item: 3257208650 r_ui = 10.00   est = 9.89   {'was_impossible': False}
user: 242        item: 3257208669 r_ui = 9.00   est = 8.94   {'was_impossible': False}
user: 242        item: 3257210450 r_ui = 10.00   est = 9.80   {'was_impossible': False}
user: 242        item: 3257212054 r_ui = 10.00   est = 9.93   {'was_impossible': False}
user: 242        item: 3257212429 r_ui = 10.00   est = 9.92   {'was_impossible': False}
user: 242        item: 3257214154 r_ui = 9.00   est = 3.81   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 242        item: 3257216416 r_ui = 7.00   

user: 254        item: 0821771949 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 254        item: 086140324X r_ui = 0.00   est = 3.81   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 254        item: 0871131811 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 254        item: 088001380X r_ui = 0.00   est = 0.01   {'was_impossible': False}
user: 254        item: 088396452X r_ui = 0.00   est = 0.02   {'was_impossible': False}
user: 254        item: 0895774011 r_ui = 0.00   est = 3.81   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 254        item: 0895775522 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 254        item: 0963094424 r_ui = 0.00   est = 0.05   {'was_impossible': False}
user: 254        item: 0965834581 r_ui = 0.00   est = 3.81   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 254        item: 1401200893 r_ui = 0.00   est = 0.03   {'was_impossible': False}
user: 254       

user: 496        item: 0425182878 r_ui = 8.00   est = 1.61   {'was_impossible': False}
user: 496        item: 055356451X r_ui = 8.00   est = 3.81   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 500        item: 0849943450 r_ui = 10.00   est = 3.81   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 500        item: 084994371X r_ui = 8.00   est = 2.24   {'was_impossible': False}
user: 501        item: 0020442408 r_ui = 0.00   est = 0.03   {'was_impossible': False}
user: 503        item: 0060987103 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 503        item: 0140179836 r_ui = 10.00   est = 9.20   {'was_impossible': False}
user: 503        item: 0310201071 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 503        item: 0312976275 r_ui = 0.00   est = 0.08   {'was_impossible': False}
user: 503        item: 0345348656 r_ui = 8.00   est = 7.46   {'was_impossible': False}
user: 503        item: 0373076746 r_ui = 7.00   est 

user: 638        item: 067972768X r_ui = 10.00   est = 10.00   {'was_impossible': False}
user: 638        item: 0679732241 r_ui = 9.00   est = 9.00   {'was_impossible': False}
user: 638        item: 0679746048 r_ui = 7.00   est = 6.98   {'was_impossible': False}
user: 638        item: 0684864320 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 638        item: 0684864843 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 638        item: 0684867818 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 638        item: 0684868911 r_ui = 0.00   est = 3.81   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 638        item: 0684874318 r_ui = 10.00   est = 9.99   {'was_impossible': False}
user: 638        item: 0688180639 r_ui = 9.00   est = 3.81   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 638        item: 0743206045 r_ui = 0.00   est = 3.81   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 638    

user: 753        item: 0679420118 r_ui = 10.00   est = 9.39   {'was_impossible': False}
user: 753        item: 0741402858 r_ui = 8.00   est = 7.25   {'was_impossible': False}
user: 753        item: 0741402912 r_ui = 10.00   est = 9.29   {'was_impossible': False}
user: 753        item: 0741403056 r_ui = 10.00   est = 3.81   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 753        item: 0741403471 r_ui = 10.00   est = 3.81   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 753        item: 0741403781 r_ui = 10.00   est = 9.23   {'was_impossible': False}
user: 753        item: 0741405784 r_ui = 8.00   est = 7.55   {'was_impossible': False}
user: 753        item: 0741407213 r_ui = 10.00   est = 9.29   {'was_impossible': False}
user: 753        item: 0741409690 r_ui = 10.00   est = 9.43   {'was_impossible': False}
user: 753        item: 074141127X r_ui = 10.00   est = 9.52   {'was_impossible': False}
user: 753        item: 0741412740 r_ui = 10.00

user: 901        item: 0451197003 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 901        item: 0486265862 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 901        item: 0551030682 r_ui = 8.00   est = 6.30   {'was_impossible': False}
user: 901        item: 0551031085 r_ui = 8.00   est = 6.14   {'was_impossible': False}
user: 901        item: 0551031123 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 901        item: 0551032510 r_ui = 9.00   est = 7.03   {'was_impossible': False}
user: 901        item: 0552140295 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 901        item: 0571197779 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 901        item: 0571200389 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 901        item: 0679745653 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 901        item: 0749322217 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 901        item: 0749394927 r_ui = 0.

user: 1083       item: 044023767X r_ui = 10.00   est = 5.51   {'was_impossible': False}
user: 1083       item: 0451526503 r_ui = 6.00   est = 3.23   {'was_impossible': False}
user: 1087       item: 0553346768 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 1087       item: 0884043142 r_ui = 9.00   est = 1.93   {'was_impossible': False}
user: 1087       item: 096478260X r_ui = 0.00   est = 0.09   {'was_impossible': False}
user: 1087       item: 155643068X r_ui = 0.00   est = 0.07   {'was_impossible': False}
user: 1095       item: 0865532168 r_ui = 10.00   est = 2.90   {'was_impossible': False}
user: 1095       item: 3478088534 r_ui = 5.00   est = 1.26   {'was_impossible': False}
user: 1096       item: 059040640X r_ui = 7.00   est = 3.81   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 1098       item: 031227999X r_ui = 7.00   est = 1.51   {'was_impossible': False}
user: 1111       item: 0316666343 r_ui = 0.00   est = 0.00   {'was_impossible': False}
use

In [32]:
#PMF 알고리즘 적용
from surprise import SVD

trainset, testset = train_test_split(data, test_size=0.25)
algo = SVD(n_factors = 100, n_epochs = 20, biased = False, lr_all = 0.005, reg_all = 0.02)

algo.fit(trainset)

uids = df['userID'].unique()
###
for i in range(1, len(uids)):
    uid = uids[i-1]
    iids = df[df.userID == uid]
    for j in range(1, len(iids)):
        iid = iids[j-1 : j].ISBN.values[0]
        r_ui = iids[j-1 : j].bookRating.values[0]
        pred = algo.predict(uid, iid, r_ui, verbose=True)

user: 8          item: 0002005018 r_ui = 5.00   est = 3.79   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 8          item: 0060973129 r_ui = 0.00   est = 0.21   {'was_impossible': False}
user: 8          item: 0374157065 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 8          item: 0393045218 r_ui = 0.00   est = 0.09   {'was_impossible': False}
user: 8          item: 0399135782 r_ui = 0.00   est = 3.79   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 8          item: 0425176428 r_ui = 0.00   est = 3.79   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 8          item: 0671870432 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 8          item: 0679425608 r_ui = 0.00   est = 3.79   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 8          item: 074322678X r_ui = 5.00   est = 1.31   {'was_impossible': False}
user: 8          item: 0771025661 r_ui = 0.00   est = 0.04   {'wa

user: 242        item: 3499110695 r_ui = 7.00   est = 6.90   {'was_impossible': False}
user: 242        item: 3499151502 r_ui = 9.00   est = 8.78   {'was_impossible': False}
user: 242        item: 349917281X r_ui = 5.00   est = 5.12   {'was_impossible': False}
user: 242        item: 3518111000 r_ui = 0.00   est = 0.03   {'was_impossible': False}
user: 242        item: 3518370324 r_ui = 10.00   est = 9.81   {'was_impossible': False}
user: 242        item: 3548254268 r_ui = 10.00   est = 9.89   {'was_impossible': False}
user: 242        item: 3570300099 r_ui = 6.00   est = 5.85   {'was_impossible': False}
user: 242        item: 3596221269 r_ui = 10.00   est = 9.78   {'was_impossible': False}
user: 242        item: 3596292646 r_ui = 7.00   est = 6.88   {'was_impossible': False}
user: 242        item: 382897239X r_ui = 0.00   est = 0.06   {'was_impossible': False}
user: 242        item: 3928812343 r_ui = 7.00   est = 6.88   {'was_impossible': False}
user: 243        item: 0060915544 r_ui =

user: 254        item: 086140324X r_ui = 0.00   est = 0.06   {'was_impossible': False}
user: 254        item: 0871131811 r_ui = 0.00   est = 3.79   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 254        item: 088001380X r_ui = 0.00   est = 0.03   {'was_impossible': False}
user: 254        item: 088396452X r_ui = 0.00   est = 0.01   {'was_impossible': False}
user: 254        item: 0895774011 r_ui = 0.00   est = 3.79   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 254        item: 0895775522 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 254        item: 0963094424 r_ui = 0.00   est = 0.01   {'was_impossible': False}
user: 254        item: 0965834581 r_ui = 0.00   est = 3.79   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 254        item: 1401200893 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 254        item: 1401201172 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 254       

user: 487        item: 0440225078 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 487        item: 0449907481 r_ui = 0.00   est = 0.25   {'was_impossible': False}
user: 487        item: 0486272664 r_ui = 9.00   est = 3.96   {'was_impossible': False}
user: 487        item: 0515127833 r_ui = 0.00   est = 0.10   {'was_impossible': False}
user: 487        item: 0515134279 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 487        item: 0553107305 r_ui = 0.00   est = 0.25   {'was_impossible': False}
user: 487        item: 0553272837 r_ui = 0.00   est = 0.11   {'was_impossible': False}
user: 487        item: 0671888587 r_ui = 6.00   est = 2.87   {'was_impossible': False}
user: 487        item: 0786015276 r_ui = 0.00   est = 3.79   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 487        item: 080213825X r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 487        item: 0945586051 r_ui = 7.00   est = 3.32   {'was_impossible': False}
user:

user: 638        item: 0553109839 r_ui = 0.00   est = 0.02   {'was_impossible': False}
user: 638        item: 0609605925 r_ui = 8.00   est = 7.92   {'was_impossible': False}
user: 638        item: 0609606727 r_ui = 8.00   est = 7.95   {'was_impossible': False}
user: 638        item: 0670031046 r_ui = 0.00   est = 3.79   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 638        item: 0670892963 r_ui = 7.00   est = 6.93   {'was_impossible': False}
user: 638        item: 0671042556 r_ui = 9.00   est = 8.92   {'was_impossible': False}
user: 638        item: 0679410430 r_ui = 10.00   est = 3.79   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 638        item: 0679419624 r_ui = 10.00   est = 9.90   {'was_impossible': False}
user: 638        item: 0679442790 r_ui = 10.00   est = 3.79   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 638        item: 0679450440 r_ui = 0.00   est = 0.02   {'was_impossible': False}
user: 638    

user: 746        item: 014036336X r_ui = 8.00   est = 3.01   {'was_impossible': False}
user: 746        item: 03453398681 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 746        item: 0345348036 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 746        item: 0375821813 r_ui = 8.00   est = 3.79   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 746        item: 0394820371 r_ui = 8.00   est = 2.62   {'was_impossible': False}
user: 746        item: 0449134482 r_ui = 0.00   est = 3.79   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 753        item: 0679420118 r_ui = 10.00   est = 9.51   {'was_impossible': False}
user: 753        item: 0741402858 r_ui = 8.00   est = 7.68   {'was_impossible': False}
user: 753        item: 0741402912 r_ui = 10.00   est = 9.76   {'was_impossible': False}
user: 753        item: 0741403056 r_ui = 10.00   est = 9.50   {'was_impossible': False}
user: 753        item: 0741403471 r_ui = 10.00   e

user: 901        item: 0380817705 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 901        item: 0451197003 r_ui = 0.00   est = 3.79   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 901        item: 0486265862 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 901        item: 0551030682 r_ui = 8.00   est = 7.15   {'was_impossible': False}
user: 901        item: 0551031085 r_ui = 8.00   est = 7.04   {'was_impossible': False}
user: 901        item: 0551031123 r_ui = 0.00   est = 0.24   {'was_impossible': False}
user: 901        item: 0551032510 r_ui = 9.00   est = 7.83   {'was_impossible': False}
user: 901        item: 0552140295 r_ui = 0.00   est = 0.02   {'was_impossible': False}
user: 901        item: 0571197779 r_ui = 0.00   est = 3.79   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 901        item: 0571200389 r_ui = 0.00   est = 3.79   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 901       

user: 1129       item: 0064407667 r_ui = 4.00   est = 1.54   {'was_impossible': False}
user: 1129       item: 014025448X r_ui = 7.00   est = 3.79   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 1129       item: 0151010102 r_ui = 7.00   est = 2.71   {'was_impossible': False}
user: 1129       item: 0312303033 r_ui = 0.00   est = 0.17   {'was_impossible': False}
user: 1129       item: 0385472951 r_ui = 6.00   est = 2.36   {'was_impossible': False}
user: 1129       item: 0385493800 r_ui = 6.00   est = 2.23   {'was_impossible': False}
user: 1129       item: 0553377876 r_ui = 3.00   est = 3.79   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 1129       item: 0618045996 r_ui = 0.00   est = 0.38   {'was_impossible': False}
user: 1129       item: 0671318314 r_ui = 7.00   est = 3.79   {'was_impossible': True, 'reason': 'User and item are unkown.'}
user: 1129       item: 1400034779 r_ui = 0.00   est = 0.02   {'was_impossible': False}
user: 1129      

In [33]:
#PMF with biased 알고리즘 적용
from surprise import SVD

trainset, testset = train_test_split(data, test_size=0.25)
algo = SVD(n_factors = 100, n_epochs = 20, biased = True, lr_all = 0.005, reg_all = 0.02)

algo.fit(trainset)

uids = df['userID'].unique()
###
for i in range(1, len(uids)):
    uid = uids[i-1]
    iids = df[df.userID == uid]
    for j in range(1, len(iids)):
        iid = iids[j-1 : j].ISBN.values[0]
        r_ui = iids[j-1 : j].bookRating.values[0]
        pred = algo.predict(uid, iid, r_ui, verbose=True)

user: 8          item: 0002005018 r_ui = 5.00   est = 3.36   {'was_impossible': False}
user: 8          item: 0060973129 r_ui = 0.00   est = 1.79   {'was_impossible': False}
user: 8          item: 0374157065 r_ui = 0.00   est = 1.72   {'was_impossible': False}
user: 8          item: 0393045218 r_ui = 0.00   est = 1.72   {'was_impossible': False}
user: 8          item: 0399135782 r_ui = 0.00   est = 2.72   {'was_impossible': False}
user: 8          item: 0425176428 r_ui = 0.00   est = 1.77   {'was_impossible': False}
user: 8          item: 0671870432 r_ui = 0.00   est = 1.88   {'was_impossible': False}
user: 8          item: 0679425608 r_ui = 0.00   est = 1.65   {'was_impossible': False}
user: 8          item: 074322678X r_ui = 5.00   est = 3.32   {'was_impossible': False}
user: 8          item: 0771025661 r_ui = 0.00   est = 1.72   {'was_impossible': False}
user: 8          item: 0771074670 r_ui = 0.00   est = 2.72   {'was_impossible': False}
user: 8          item: 080652121X r_ui = 0.

user: 233        item: 9511173758 r_ui = 9.00   est = 5.34   {'was_impossible': False}
user: 237        item: 0785263292 r_ui = 0.00   est = 2.38   {'was_impossible': False}
user: 242        item: 0553278223 r_ui = 10.00   est = 8.85   {'was_impossible': False}
user: 242        item: 0971880107 r_ui = 0.00   est = 4.60   {'was_impossible': False}
user: 242        item: 3150000335 r_ui = 10.00   est = 8.18   {'was_impossible': False}
user: 242        item: 3257202083 r_ui = 10.00   est = 6.21   {'was_impossible': False}
user: 242        item: 3257203136 r_ui = 8.00   est = 7.16   {'was_impossible': False}
user: 242        item: 3257203659 r_ui = 9.00   est = 7.66   {'was_impossible': False}
user: 242        item: 3257207522 r_ui = 10.00   est = 8.80   {'was_impossible': False}
user: 242        item: 3257208626 r_ui = 8.00   est = 7.37   {'was_impossible': False}
user: 242        item: 3257208634 r_ui = 10.00   est = 8.53   {'was_impossible': False}
user: 242        item: 3257208640 r_ui

user: 254        item: 0743225406 r_ui = 0.00   est = 0.14   {'was_impossible': False}
user: 254        item: 0743245024 r_ui = 0.00   est = 0.28   {'was_impossible': False}
user: 254        item: 0749748001 r_ui = 0.00   est = 1.62   {'was_impossible': False}
user: 254        item: 0749748028 r_ui = 0.00   est = 0.17   {'was_impossible': False}
user: 254        item: 076072850X r_ui = 0.00   est = 0.01   {'was_impossible': False}
user: 254        item: 0765300222 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 254        item: 0801957729 r_ui = 0.00   est = 0.13   {'was_impossible': False}
user: 254        item: 0804106304 r_ui = 0.00   est = 1.27   {'was_impossible': False}
user: 254        item: 080410753X r_ui = 0.00   est = 0.34   {'was_impossible': False}
user: 254        item: 0804108552 r_ui = 0.00   est = 0.11   {'was_impossible': False}
user: 254        item: 0808520695 r_ui = 0.00   est = 1.62   {'was_impossible': False}
user: 254        item: 0811826597 r_ui = 0.

user: 486        item: 0060976845 r_ui = 8.00   est = 6.06   {'was_impossible': False}
user: 486        item: 0156027321 r_ui = 0.00   est = 5.53   {'was_impossible': False}
user: 486        item: 0375403582 r_ui = 0.00   est = 4.73   {'was_impossible': False}
user: 486        item: 0385721420 r_ui = 0.00   est = 4.73   {'was_impossible': False}
user: 486        item: 0451202341 r_ui = 8.00   est = 5.31   {'was_impossible': False}
user: 487        item: 0060913509 r_ui = 0.00   est = 2.10   {'was_impossible': False}
user: 487        item: 0060923245 r_ui = 0.00   est = 1.00   {'was_impossible': False}
user: 487        item: 0140286276 r_ui = 0.00   est = 2.10   {'was_impossible': False}
user: 487        item: 0312145543 r_ui = 0.00   est = 2.10   {'was_impossible': False}
user: 487        item: 0312980388 r_ui = 0.00   est = 1.17   {'was_impossible': False}
user: 487        item: 0340695617 r_ui = 0.00   est = 1.13   {'was_impossible': False}
user: 487        item: 0345441133 r_ui = 7.

user: 638        item: 0140128107 r_ui = 10.00   est = 6.48   {'was_impossible': False}
user: 638        item: 0151000719 r_ui = 10.00   est = 6.48   {'was_impossible': False}
user: 638        item: 0151001006 r_ui = 10.00   est = 9.76   {'was_impossible': False}
user: 638        item: 0316603287 r_ui = 9.00   est = 8.73   {'was_impossible': False}
user: 638        item: 0316666343 r_ui = 10.00   est = 9.37   {'was_impossible': False}
user: 638        item: 0316693006 r_ui = 10.00   est = 9.25   {'was_impossible': False}
user: 638        item: 0316693200 r_ui = 9.00   est = 8.94   {'was_impossible': False}
user: 638        item: 0316769487 r_ui = 10.00   est = 9.43   {'was_impossible': False}
user: 638        item: 0316776963 r_ui = 10.00   est = 6.87   {'was_impossible': False}
user: 638        item: 0316779423 r_ui = 10.00   est = 9.63   {'was_impossible': False}
user: 638        item: 0316779490 r_ui = 9.00   est = 6.48   {'was_impossible': False}
user: 638        item: 0316789089 r

user: 741        item: 0399206868 r_ui = 0.00   est = 0.36   {'was_impossible': False}
user: 741        item: 0425124347 r_ui = 0.00   est = 0.61   {'was_impossible': False}
user: 741        item: 0425139069 r_ui = 0.00   est = 0.38   {'was_impossible': False}
user: 741        item: 0425147517 r_ui = 0.00   est = 2.42   {'was_impossible': False}
user: 741        item: 0425155404 r_ui = 0.00   est = 0.89   {'was_impossible': False}
user: 741        item: 0425166619 r_ui = 0.00   est = 0.87   {'was_impossible': False}
user: 741        item: 044022165X r_ui = 0.00   est = 0.06   {'was_impossible': False}
user: 741        item: 044132231X r_ui = 8.00   est = 6.77   {'was_impossible': False}
user: 741        item: 0446364762 r_ui = 0.00   est = 0.25   {'was_impossible': False}
user: 741        item: 0450029956 r_ui = 10.00   est = 8.15   {'was_impossible': False}
user: 741        item: 0451524489 r_ui = 10.00   est = 8.25   {'was_impossible': False}
user: 741        item: 0451526503 r_ui = 

user: 882        item: 0553278223 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 882        item: 0553280589 r_ui = 10.00   est = 9.79   {'was_impossible': False}
user: 882        item: 0553290207 r_ui = 10.00   est = 9.79   {'was_impossible': False}
user: 882        item: 0553561278 r_ui = 0.00   est = 0.12   {'was_impossible': False}
user: 882        item: 0553569058 r_ui = 0.00   est = 0.09   {'was_impossible': False}
user: 882        item: 0553569074 r_ui = 0.00   est = 0.21   {'was_impossible': False}
user: 882        item: 0553571656 r_ui = 0.00   est = 0.10   {'was_impossible': False}
user: 882        item: 0553572997 r_ui = 10.00   est = 9.77   {'was_impossible': False}
user: 882        item: 0553801945 r_ui = 0.00   est = 3.03   {'was_impossible': False}
user: 882        item: 0590445561 r_ui = 0.00   est = 3.03   {'was_impossible': False}
user: 882        item: 0618260307 r_ui = 10.00   est = 9.89   {'was_impossible': False}
user: 882        item: 067081976X r_ui 

user: 1031       item: 0345458915 r_ui = 7.00   est = 4.89   {'was_impossible': False}
user: 1032       item: 006092988X r_ui = 8.00   est = 4.35   {'was_impossible': False}
user: 1032       item: 038531258X r_ui = 10.00   est = 4.35   {'was_impossible': False}
user: 1048       item: 0553256696 r_ui = 9.00   est = 6.23   {'was_impossible': False}
user: 1048       item: 0785263780 r_ui = 8.00   est = 5.93   {'was_impossible': False}
user: 1054       item: 0375413081 r_ui = 9.00   est = 5.65   {'was_impossible': False}
user: 1057       item: 038533303X r_ui = 6.00   est = 4.05   {'was_impossible': False}
user: 1057       item: 0440414520 r_ui = 0.00   est = 2.90   {'was_impossible': False}
user: 1063       item: 0345387007 r_ui = 5.00   est = 4.72   {'was_impossible': False}
user: 1063       item: 0385247745 r_ui = 0.00   est = 4.72   {'was_impossible': False}
user: 1063       item: 0385293542 r_ui = 0.00   est = 3.17   {'was_impossible': False}
user: 1063       item: 0671311891 r_ui = 9

user: 1248       item: 0803281730 r_ui = 10.00   est = 9.29   {'was_impossible': False}
user: 1248       item: 0805051732 r_ui = 9.00   est = 8.88   {'was_impossible': False}
user: 1248       item: 0811807150 r_ui = 9.00   est = 8.84   {'was_impossible': False}
user: 1248       item: 0934257345 r_ui = 9.00   est = 7.93   {'was_impossible': False}
user: 1248       item: 1558534245 r_ui = 10.00   est = 9.27   {'was_impossible': False}
user: 1248       item: 1560980087 r_ui = 10.00   est = 9.19   {'was_impossible': False}
user: 1249       item: 0440226430 r_ui = 5.00   est = 4.85   {'was_impossible': False}
user: 1249       item: 0449300021 r_ui = 0.00   est = 4.70   {'was_impossible': False}
user: 1249       item: 0671004565 r_ui = 8.00   est = 5.78   {'was_impossible': False}
user: 1249       item: 0684835983 r_ui = 8.00   est = 4.36   {'was_impossible': False}


In [34]:
#SVDpp 알고리즘 적용
from surprise import SVDpp

trainset, testset = train_test_split(data, test_size=0.25)
algo = SVDpp(n_factors = 100, n_epochs = 20, lr_all = 0.005, reg_all = 0.02)

algo.fit(trainset)

uids = df['userID'].unique()
###
for i in range(1, len(uids)):
    uid = uids[i-1]
    iids = df[df.userID == uid]
    for j in range(1, len(iids)):
        iid = iids[j-1 : j].ISBN.values[0]
        r_ui = iids[j-1 : j].bookRating.values[0]
        pred = algo.predict(uid, iid, r_ui, verbose=True)

user: 8          item: 0002005018 r_ui = 5.00   est = 3.83   {'was_impossible': False}
user: 8          item: 0060973129 r_ui = 0.00   est = 2.78   {'was_impossible': False}
user: 8          item: 0374157065 r_ui = 0.00   est = 1.08   {'was_impossible': False}
user: 8          item: 0393045218 r_ui = 0.00   est = 2.78   {'was_impossible': False}
user: 8          item: 0399135782 r_ui = 0.00   est = 1.18   {'was_impossible': False}
user: 8          item: 0425176428 r_ui = 0.00   est = 1.05   {'was_impossible': False}
user: 8          item: 0671870432 r_ui = 0.00   est = 1.10   {'was_impossible': False}
user: 8          item: 0679425608 r_ui = 0.00   est = 1.53   {'was_impossible': False}
user: 8          item: 074322678X r_ui = 5.00   est = 3.63   {'was_impossible': False}
user: 8          item: 0771025661 r_ui = 0.00   est = 2.78   {'was_impossible': False}
user: 8          item: 0771074670 r_ui = 0.00   est = 2.78   {'was_impossible': False}
user: 8          item: 080652121X r_ui = 0.

user: 183        item: 9729720495 r_ui = 9.00   est = 8.92   {'was_impossible': False}
user: 183        item: 972975831X r_ui = 7.00   est = 7.00   {'was_impossible': False}
user: 183        item: 9729817715 r_ui = 6.00   est = 5.95   {'was_impossible': False}
user: 183        item: 9729852405 r_ui = 8.00   est = 7.94   {'was_impossible': False}
user: 185        item: 0380778556 r_ui = 10.00   est = 7.15   {'was_impossible': False}
user: 199        item: 0373196407 r_ui = 0.00   est = 3.05   {'was_impossible': False}
user: 199        item: 0373196520 r_ui = 0.00   est = 1.41   {'was_impossible': False}
user: 199        item: 0385336772 r_ui = 0.00   est = 1.20   {'was_impossible': False}
user: 199        item: 0440949424 r_ui = 0.00   est = 3.05   {'was_impossible': False}
user: 204        item: 0451452755 r_ui = 0.00   est = 1.88   {'was_impossible': False}
user: 228        item: 0385337639 r_ui = 7.00   est = 5.57   {'was_impossible': False}
user: 232        item: 0007123817 r_ui = 7

user: 254        item: 0375713549 r_ui = 0.00   est = 0.07   {'was_impossible': False}
user: 254        item: 0375724370 r_ui = 0.00   est = 0.06   {'was_impossible': False}
user: 254        item: 0375727345 r_ui = 0.00   est = 0.07   {'was_impossible': False}
user: 254        item: 0375751513 r_ui = 0.00   est = 1.81   {'was_impossible': False}
user: 254        item: 0375756469 r_ui = 9.00   est = 1.81   {'was_impossible': False}
user: 254        item: 0375801677 r_ui = 0.00   est = 1.81   {'was_impossible': False}
user: 254        item: 0375814248 r_ui = 0.00   est = 0.08   {'was_impossible': False}
user: 254        item: 0375823352 r_ui = 0.00   est = 0.07   {'was_impossible': False}
user: 254        item: 037582345X r_ui = 9.00   est = 8.79   {'was_impossible': False}
user: 254        item: 0375823468 r_ui = 9.00   est = 8.85   {'was_impossible': False}
user: 254        item: 038070174X r_ui = 0.00   est = 0.06   {'was_impossible': False}
user: 254        item: 0380730448 r_ui = 8.

user: 383        item: 0140374248 r_ui = 0.00   est = 0.19   {'was_impossible': False}
user: 383        item: 0142001740 r_ui = 0.00   est = 0.35   {'was_impossible': False}
user: 383        item: 0195086295 r_ui = 0.00   est = 0.00   {'was_impossible': False}
user: 383        item: 0312995423 r_ui = 0.00   est = 2.40   {'was_impossible': False}
user: 383        item: 0380704587 r_ui = 0.00   est = 0.14   {'was_impossible': False}
user: 383        item: 0380815923 r_ui = 10.00   est = 8.71   {'was_impossible': False}
user: 383        item: 0385416342 r_ui = 10.00   est = 8.68   {'was_impossible': False}
user: 383        item: 0385418493 r_ui = 0.00   est = 0.17   {'was_impossible': False}
user: 383        item: 038542471X r_ui = 0.00   est = 0.21   {'was_impossible': False}
user: 383        item: 0425098109 r_ui = 5.00   est = 4.62   {'was_impossible': False}
user: 383        item: 0449005410 r_ui = 0.00   est = 0.36   {'was_impossible': False}
user: 383        item: 0449202631 r_ui = 

user: 505        item: 0553202294 r_ui = 8.00   est = 7.20   {'was_impossible': False}
user: 505        item: 0553225987 r_ui = 0.00   est = 1.60   {'was_impossible': False}
user: 505        item: 067170124X r_ui = 0.00   est = 1.56   {'was_impossible': False}
user: 505        item: 0671868713 r_ui = 8.00   est = 7.08   {'was_impossible': False}
user: 505        item: 0679401377 r_ui = 8.00   est = 4.85   {'was_impossible': False}
user: 505        item: 0805056491 r_ui = 9.00   est = 7.91   {'was_impossible': False}
user: 505        item: 0816152446 r_ui = 7.00   est = 6.40   {'was_impossible': False}
user: 507        item: 0020427859 r_ui = 9.00   est = 8.85   {'was_impossible': False}
user: 507        item: 0060392452 r_ui = 0.00   est = 0.02   {'was_impossible': False}
user: 507        item: 0060558865 r_ui = 8.00   est = 7.94   {'was_impossible': False}
user: 507        item: 0060575808 r_ui = 0.00   est = 0.03   {'was_impossible': False}
user: 507        item: 0060934417 r_ui = 0.

...

user: 753        item: 0741405784 r_ui = 8.00   est = 7.89   {'was_impossible': False}
user: 753        item: 0741407213 r_ui = 10.00   est = 9.59   {'was_impossible': False}
user: 753        item: 0741409690 r_ui = 10.00   est = 6.76   {'was_impossible': False}
user: 753        item: 074141127X r_ui = 10.00   est = 6.76   {'was_impossible': False}
user: 753        item: 0741412740 r_ui = 10.00   est = 6.76   {'was_impossible': False}
user: 753        item: 0759612277 r_ui = 10.00   est = 9.37   {'was_impossible': False}
user: 753        item: 0871137380 r_ui = 10.00   est = 9.33   {'was_impossible': False}
user: 753        item: 1585007366 r_ui = 10.00   est = 6.76   {'was_impossible': False}
user: 753        item: 1588206769 r_ui = 10.00   est = 9.40   {'was_impossible': False}
user: 758        item: 0140620109 r_ui = 9.00   est = 7.05   {'was_impossible': False}
user: 758        item: 0140620125 r_ui = 10.00   est = 7.97   {'was_impossible': False}
user: 758        item: 0375420827 

user: 882        item: 1904301088 r_ui = 0.00   est = 0.07   {'was_impossible': False}
user: 885        item: 015644450X r_ui = 0.00   est = 1.75   {'was_impossible': False}
user: 885        item: 0380002450 r_ui = 9.00   est = 6.45   {'was_impossible': False}
user: 896        item: 0140620338 r_ui = 8.00   est = 5.88   {'was_impossible': False}
user: 899        item: 0971880107 r_ui = 2.00   est = 1.74   {'was_impossible': False}
user: 899        item: 3257061269 r_ui = 9.00   est = 7.23   {'was_impossible': False}
user: 899        item: 3404126661 r_ui = 0.00   est = 0.90   {'was_impossible': False}
user: 899        item: 3453088433 r_ui = 4.00   est = 3.63   {'was_impossible': False}
user: 899        item: 3453177622 r_ui = 0.00   est = 1.17   {'was_impossible': False}
user: 899        item: 3453199766 r_ui = 8.00   est = 6.63   {'was_impossible': False}
user: 899        item: 3453210999 r_ui = 0.00   est = 1.28   {'was_impossible': False}
user: 899        item: 3453863593 r_ui = 0.

user: 1025       item: 0575073004 r_ui = 8.00   est = 7.90   {'was_impossible': False}
user: 1025       item: 0575073314 r_ui = 8.00   est = 5.14   {'was_impossible': False}
user: 1025       item: 058242092X r_ui = 7.00   est = 5.14   {'was_impossible': False}
user: 1025       item: 0586210687 r_ui = 9.00   est = 5.14   {'was_impossible': False}
user: 1025       item: 0590112899 r_ui = 0.00   est = 0.21   {'was_impossible': False}
user: 1025       item: 0590542443 r_ui = 0.00   est = 5.14   {'was_impossible': False}
user: 1025       item: 0590660543 r_ui = 0.00   est = 0.03   {'was_impossible': False}
user: 1025       item: 0603554261 r_ui = 0.00   est = 5.14   {'was_impossible': False}
user: 1025       item: 0613117905 r_ui = 9.00   est = 8.89   {'was_impossible': False}
user: 1025       item: 0679730346 r_ui = 0.00   est = 0.07   {'was_impossible': False}
user: 1025       item: 0679883878 r_ui = 8.00   est = 7.96   {'was_impossible': False}
user: 1025       item: 068484267X r_ui = 9.

user: 1178       item: 014056232X r_ui = 10.00   est = 3.88   {'was_impossible': False}
user: 1178       item: 0440203155 r_ui = 10.00   est = 7.41   {'was_impossible': False}
user: 1178       item: 0440401119 r_ui = 9.00   est = 6.85   {'was_impossible': False}
user: 1178       item: 0440940001 r_ui = 2.00   est = 2.81   {'was_impossible': False}
user: 1178       item: 0590317822 r_ui = 0.00   est = 1.84   {'was_impossible': False}
user: 1184       item: 0061056103 r_ui = 0.00   est = 0.91   {'was_impossible': False}
user: 1184       item: 034536628X r_ui = 7.00   est = 5.69   {'was_impossible': False}
user: 1184       item: 0425157539 r_ui = 0.00   est = 1.09   {'was_impossible': False}
user: 1184       item: 0425163385 r_ui = 7.00   est = 5.59   {'was_impossible': False}
user: 1184       item: 0553585118 r_ui = 3.00   est = 2.61   {'was_impossible': False}
user: 1184       item: 0553585681 r_ui = 0.00   est = 0.98   {'was_impossible': False}
user: 1184       item: 0671042262 r_ui = 

user: 1248       item: 0609601628 r_ui = 10.00   est = 9.93   {'was_impossible': False}
user: 1248       item: 0670867144 r_ui = 10.00   est = 7.12   {'was_impossible': False}
user: 1248       item: 0679412271 r_ui = 9.00   est = 8.97   {'was_impossible': False}
user: 1248       item: 0679452281 r_ui = 9.00   est = 8.92   {'was_impossible': False}
user: 1248       item: 0785324712 r_ui = 10.00   est = 9.68   {'was_impossible': False}
user: 1248       item: 0803281730 r_ui = 10.00   est = 9.47   {'was_impossible': False}
user: 1248       item: 0805051732 r_ui = 9.00   est = 8.93   {'was_impossible': False}
user: 1248       item: 0811807150 r_ui = 9.00   est = 8.76   {'was_impossible': False}
user: 1248       item: 0934257345 r_ui = 9.00   est = 9.03   {'was_impossible': False}
user: 1248       item: 1558534245 r_ui = 10.00   est = 9.78   {'was_impossible': False}
user: 1248       item: 1560980087 r_ui = 10.00   est = 7.12   {'was_impossible': False}
user: 1249       item: 0440226430 r_u